In [1]:
import heapq

# State: (M_left, C_left, boat_position)
# boat_position: 'L' left, 'R' right
START = (3, 3, 'L')
GOAL = (0, 0, 'R')

# Possible moves: (M_to_move, C_to_move) - boat capacity is 2
MOVES = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]

def is_valid_state(m_left, c_left):
    """Checks if the state is valid (no missionaries outnumbered by cannibals)."""
    m_right = 3 - m_left
    c_right = 3 - c_left

    # Numbers should be within valid range
    if not (0 <= m_left <= 3 and 0 <= c_left <= 3):
        return False

    # Missionaries should not be outnumbered on either side
    # Condition: If there are missionaries, cannibals must not outnumber them.
    if m_left > 0 and c_left > m_left:
        return False
    if m_right > 0 and c_right > m_right:
        return False
    
    return True

def heuristic(state):
    """Admissible heuristic: total number of people remaining on the left bank."""
    m_left, c_left, boat = state
    # This heuristic is admissible because the total number of people on the left 
    # must be moved to the right, and in the best case, two people move per trip.
    return m_left + c_left

def get_successors(state):
    """Generates all valid successor states from the current state."""
    m_left, c_left, boat = state
    successors = []
    
    for m_move, c_move in MOVES:
        if boat == 'L':
            # Moving from Left to Right (subtract from left)
            new_m_left = m_left - m_move
            new_c_left = c_left - c_move
            new_boat = 'R'
        else:
            # Moving from Right to Left (add to left)
            new_m_left = m_left + m_move
            new_c_left = c_left + c_move
            new_boat = 'L'
        
        new_state = (new_m_left, new_c_left, new_boat)

        # Validate state (only checking M/C counts on the left bank is sufficient due to M+C=3)
        if is_valid_state(new_m_left, new_c_left):
            successors.append(new_state)
            
    return successors

def a_star_search(start, goal):
    """Performs A* search to find a solution path."""
    # (f, g, state, path)
    open_list = []
    # The original code had '0' for g, which is correct for the start node. 
    # Corrected path to be [start] instead of [] in the tuple as the original code's path tracking was confusing.
    # The original code's push was: (heuristic(start), 0, start, []).
    # The original code's pop was: f, g, state, path = heappop(open_list)
    # The original code's path append was: path + [state] 
    # I'll modify the path to store the sequence of *states* for clarity, and g is the path cost.
    heapq.heappush(open_list, (heuristic(start), 0, start, [start])) 
    visited = set()
    
    while open_list:
        # Pop the state with the lowest f = g + h
        f, g, state, path = heapq.heappop(open_list) 
        
        if state in visited:
            continue
        visited.add(state)

        if state == goal:
            return path # Found the sequence of states
        
        for succ in get_successors(state):
            if succ not in visited:
                g_new = g + 1
                f_new = g_new + heuristic(succ)
                new_path = path + [succ]
                heapq.heappush(open_list, (f_new, g_new, succ, new_path))

    return None

def display_path(path):
    """Prints the solution path clearly."""
    print("\nSolution path (M_left, C_left, Boat_Position):")
    for i, state in enumerate(path):
        print(f"Step {i}: {state}")

# Driver code
if __name__ == "__main__":
    path = a_star_search(START, GOAL)
    if path:
        display_path(path)
    else:
        print("No solution found.")


Solution path (M_left, C_left, Boat_Position):
Step 0: (3, 3, 'L')
Step 1: (2, 2, 'R')
Step 2: (3, 2, 'L')
Step 3: (3, 0, 'R')
Step 4: (3, 1, 'L')
Step 5: (1, 1, 'R')
Step 6: (2, 2, 'L')
Step 7: (0, 2, 'R')
Step 8: (0, 3, 'L')
Step 9: (0, 1, 'R')
Step 10: (0, 2, 'L')
Step 11: (0, 0, 'R')
